## 📊 Pandas vs Spark: Equivalenze Base

| Operazione | Pandas | Spark |
|-----------|--------|-------|
| **Leggere CSV** | `pd.read_csv("file.csv")` | `spark.read.csv("file.csv", header=True, inferSchema=True)` |
| **Prime righe** | `df.head()` | `df.show()` |
| **Info struttura** | `df.info()` | `df.printSchema()` |
| **Contare righe** | `len(df)` | `df.count()` |
| **Statistiche** | `df.describe()` | `df.describe().show()` |
| **Filtrare** | `df[df['age'] > 18]` | `df.filter(df.age > 18)` |
| **Selezionare colonne** | `df[['col1', 'col2']]` | `df.select('col1', 'col2')` |
| **Nuova colonna** | `df['new'] = df['old'] * 2` | `df.withColumn('new', col('old') * 2)` |
| **Ordinare** | `df.sort_values('age')` | `df.orderBy('age')` |
| **Group by** | `df.groupby('category').sum()` | `df.groupBy('category').sum()` |
| **Join** | `df1.merge(df2, on='id')` | `df1.join(df2, 'id')` |

### ⚠️ DIFFERENZA CHIAVE: Spark è immutabile!

```python
# ❌ PANDAS: Modifica in-place
df['new_col'] = df['old_col'] * 2  # Modifica df

# ✅ SPARK: Crea nuovo DataFrame
df_new = df.withColumn('new_col', col('old_col') * 2)  # df rimane uguale!
```

## 🎯 Esempio Pratico 1: Caricamento e Esplorazione

### Con Pandas (su dati piccoli):
```python
import pandas as pd

df = pd.read_csv("customers.csv")
print(df.head())
print(df.shape)
print(df['age'].mean())
```

### Con Spark (su dati grandi):

In [ ]:
# Spark legge da file o tabelle
df = spark.read.csv("/path/to/customers.csv", header=True, inferSchema=True)

# Esplora struttura
df.printSchema()  # Come df.info() in Pandas

# Prime righe
df.show(5)  # Come df.head() in Pandas

# Conta righe
print(f"Totale righe: {df.count()}")  # Come len(df)

# Statistiche
df.describe().show()  # Come df.describe()

## 🔄 Esempio Pratico 2: Trasformazioni

### Con Pandas:
```python
# Filtro
df_filtered = df[df['age'] > 25]

# Nuova colonna
df['age_group'] = df['age'].apply(lambda x: 'Senior' if x > 60 else 'Adult')

# Seleziona colonne
df_subset = df[['name', 'age', 'age_group']]
```

### Con Spark:

In [ ]:
from pyspark.sql.functions import col, when

# Filtro (come Pandas)
df_filtered = df.filter(col('age') > 25)

# Nuova colonna con logica condizionale
df_transformed = df.withColumn('age_group', 
    when(col('age') > 60, 'Senior')
    .otherwise('Adult')
)

# Seleziona colonne
df_subset = df_transformed.select('name', 'age', 'age_group')

# Mostra risultato
df_subset.show()

## 📊 Esempio Pratico 3: Aggregazioni

### Con Pandas:
```python
# Group by base
df.groupby('category')['amount'].sum()

# Aggregazioni multiple
df.groupby('category').agg({
    'amount': ['sum', 'mean'],
    'customer_id': 'count'
})
```

### Con Spark:

In [ ]:
from pyspark.sql.functions import sum, avg, count

# Group by base
df.groupBy('category').sum('amount').show()

# Aggregazioni multiple (come Pandas .agg())
df.groupBy('category').agg(
    sum('amount').alias('total_amount'),
    avg('amount').alias('avg_amount'),
    count('customer_id').alias('num_customers')
).show()

## 🔗 Esempio Pratico 4: Join (come merge in Pandas)

### Con Pandas:
```python
# Inner join
result = df1.merge(df2, on='customer_id', how='inner')

# Left join
result = df1.merge(df2, on='customer_id', how='left')
```

### Con Spark:

In [ ]:
# Supponiamo di avere due tabelle: customers e orders

customers = spark.read.table("customers")
orders = spark.read.table("orders")

# Inner join (default)
result = customers.join(orders, 'customer_id', 'inner')

# Left join
result = customers.join(orders, 'customer_id', 'left')

# Join con colonne diverse
result = customers.join(orders, customers.id == orders.customer_id, 'inner')

result.show()

## 💾 Esempio Pratico 5: Salvare Risultati

### Con Pandas:
```python
# Salva CSV
df.to_csv('output.csv', index=False)

# Salva Parquet
df.to_parquet('output.parquet')
```

### Con Spark:

In [ ]:
# Salva come tabella (modo più comune in Databricks)
df.write.mode('overwrite').saveAsTable('my_database.results')

# Salva come file Parquet
df.write.mode('overwrite').parquet('/path/to/output.parquet')

# Salva CSV
df.write.mode('overwrite').csv('/path/to/output.csv', header=True)

## 🗄️ Databricks Specifico: Unity Catalog

In Databricks, i dati sono organizzati in modo gerarchico:

```
Catalog (come un database server)
  └─ Schema (come un database)
      └─ Tabelle
```

### Creare struttura:

In [ ]:
# Crea catalog
spark.sql("CREATE CATALOG IF NOT EXISTS my_catalog")

# Crea schema (database)
spark.sql("CREATE SCHEMA IF NOT EXISTS my_catalog.my_schema")

# Usa lo schema
spark.sql("USE my_catalog.my_schema")

# Salva tabella
df.write.saveAsTable("my_catalog.my_schema.my_table")

# Leggi tabella
df = spark.read.table("my_catalog.my_schema.my_table")

## 🚀 Esempio Completo: Pipeline ETL Base

### Template standard per ogni esercizio Databricks:

In [ ]:
from pyspark.sql.functions import col, when, sum, avg, count

# ========== 1. ESTRAZIONE (Extract) ==========
# Leggi dati da file o tabella
df_raw = spark.read.csv("/path/to/data.csv", header=True, inferSchema=True)

# ========== 2. ESPLORAZIONE ==========
print("Schema:")
df_raw.printSchema()

print("\nPrime 5 righe:")
df_raw.show(5)

print(f"\nTotale righe: {df_raw.count()}")

# ========== 3. TRASFORMAZIONE (Transform) ==========
# Pulizia
df_clean = df_raw.filter(col('amount').isNotNull())

# Nuove colonne
df_transformed = df_clean.withColumn('amount_category',
    when(col('amount') > 1000, 'High')
    .when(col('amount') > 100, 'Medium')
    .otherwise('Low')
)

# ========== 4. AGGREGAZIONE ==========
df_result = df_transformed.groupBy('category').agg(
    count('*').alias('total_records'),
    sum('amount').alias('total_amount'),
    avg('amount').alias('avg_amount')
)

# ========== 5. CARICAMENTO (Load) ==========
# Mostra risultato
df_result.show()

# Salva tabella
df_result.write.mode('overwrite').saveAsTable('my_catalog.my_schema.results')

## ⚠️ Errori Comuni da Evitare

```python
# ❌ SBAGLIATO: Porta tutto in memoria su una sola macchina
df_pandas = df.toPandas()
df_pandas.groupby('col').sum()

# ✅ CORRETTO: Usa Spark distribuito
df.groupBy('col').sum().show()
```

### 2. ❌ NON iterare riga per riga!
```python
# ❌ SBAGLIATO: Lentissimo
for row in df.collect():
    process_row(row)

# ✅ CORRETTO: Usa trasformazioni vettoriali
df.withColumn('new_col', col('old_col') * 2)
```

### 3. ⚠️ `.show()` vs `.collect()` vs `.display()`
```python
df.show()      # Mostra prime 20 righe (sicuro)
df.display()   # Databricks-only: visualizzazione interattiva (sicuro)
df.collect()   # Porta TUTTO in memoria (pericoloso con big data!)
```

## 🔥 Best Practices

### 1. **Lazy Evaluation**: Spark costruisce un piano, esegue solo quando necessario
```python
# Questi NON vengono eseguiti subito
df1 = df.filter(col('age') > 18)
df2 = df1.withColumn('new_col', col('old_col') * 2)
df3 = df2.groupBy('category').count()

# Esegue SOLO qui (Action)
df3.show()  # Ora Spark esegue tutto insieme, ottimizzato
```

### 2. **Cache per riutilizzare dati**
```python
# Se usi lo stesso DataFrame molte volte
df_filtered = df.filter(col('age') > 18).cache()

df_filtered.count()  # Esegue e salva in memoria
df_filtered.show()   # Usa cache, non ricalcola
```

### 3. **Usa SQL quando è più chiaro**
```python
# Registra come vista temporanea
df.createOrReplaceTempView('my_data')

# Usa SQL (utile se conosci bene SQL)
result = spark.sql("""
    SELECT category, COUNT(*) as count, AVG(amount) as avg_amount
    FROM my_data
    WHERE amount > 100
    GROUP BY category
    ORDER BY avg_amount DESC
""")
result.show()
```

## 📚 Prossimi Passi

Ora che conosci le basi, esplora:

1. **[databricks_cheatsheet.ipynb](databricks_cheatsheet.ipynb)** - Comandi ordinati per complessità
2. **[PRATICA.ipynb](../PRATICA.ipynb)** - Esercizi pratici con Delta Live Tables
3. **[LEZIONI/](../LEZIONI/)** - Approfondimenti su architettura e concetti

### 🎯 Cosa fare per imparare:
1. Prendi un tuo progetto Pandas esistente
2. Prova a riscriverlo in Spark usando questo notebook come riferimento
3. Sperimenta con dataset più grandi (dove Pandas non ce la fa)

### 🆘 Risorse utili:
- [Databricks Community Edition](https://community.cloud.databricks.com/) - Gratis per imparare
- [PySpark Documentation](https://spark.apache.org/docs/latest/api/python/)
- [Pandas to Spark Cheat Sheet](https://sparkbyexamples.com/pandas/pandas-vs-pyspark/)